In [1]:
from math import log
import operator

In [2]:
def createDataSet():
    #  房    车
    dataSet=[[1,1,'yes'],
             [1,1,'yes'],
             [1,0,'yes'],
             [0,1,'no'],
             [0,1,'no'],
             [0,0,'no'],
             [0,0,'yes'],
             [0,0,'no']]
    labels=['是否有房产','是否有车']
    return dataSet,labels

In [3]:
dataset,labels=createDataSet()
print(dataset)
print(labels)

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'yes'], [0, 1, 'no'], [0, 1, 'no'], [0, 0, 'no'], [0, 0, 'yes'], [0, 0, 'no']]
['是否有房产', '是否有车']


ID-> 熵  衡量数据的纯度
 计算公式
 计算条件熵
 计算  信息增益=信息熵-条件熵

In [4]:
# 计算信息熵   pi log(pi,2)
def calcShannonEnt(dataset):
    shannonEnt=0.0
    num=len(dataset)
    #字典
    labelCounts={}  # {'yes':2,'no':3}
    for item in dataset:
        label=item[-1]
        if label not in labelCounts:  #在字典中这个label是否存在
            labelCounts[label]=0
        labelCounts[label]+=1
    
    #计算熵
    for key in labelCounts:
        p=float(labelCounts[key]/num)
        shannonEnt -= p*log(p,2)
    return shannonEnt

In [5]:
#测试这个数据集的熵
shan=calcShannonEnt(dataset)
print('整个数据集的信息熵：',shan)
'''
熵越高说明信息混合的数据越多
'''
#追加一个分类
dataset[7][-1]='不确定'
shan=calcShannonEnt(dataset)
print('整个数据集的信息熵：',shan)

整个数据集的信息熵： 1.0
整个数据集的信息熵： 1.4056390622295662


In [5]:

'''
另一种 度量信息无序程度的方法是gini  它指的是从一个数据集中随机选取一个子项，度量其被错误分类其他分组的概率

条件   axis:0        value=1
返回   [[1,'yes'],[1,'yes']]

条件   axis:0        value=1


条件   axis:0        value=1


条件   axis:0        value=1


'''
def splitDataSet(dataset,axis,value):
    result=[]
    for item in dataset:
        if item[axis]==value:
            r=item[:axis]+item[axis+1:]   # []+[1,'yes'] => [1,'yes']
            result.append(r)
    return result
dataset,labels=createDataSet()

result=splitDataSet(dataset,0,1)
print(result)

result=splitDataSet(dataset,0,0)
print(result)

result=splitDataSet(dataset,1,1)
print(result)

result=splitDataSet(dataset,1,0)
print(result)

[[1, 'yes'], [1, 'yes'], [0, 'yes']]
[[1, 'no'], [1, 'no'], [0, 'no'], [0, 'yes'], [0, 'no']]
[[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]
[[1, 'yes'], [0, 'no'], [0, 'yes'], [0, 'no']]


In [6]:
itema=[1, 0, 'yes']
print(itema[:1]+itema[2:])
#print(dataset)

[1, 'yes']


In [9]:
'''
寻找最佳特征：
    piH(Y|x=xi)
'''
def chooseBestFeatures(dataset):
    #特征总数   2
    numFeatures=len(dataset[0])-1  #标签不是特征  是结果
    #计算信息熵  1
    ent=calcShannonEnt(dataset)
    #最佳特征列的信息熵  yi
    bestGain=0.0
    # 最佳特征号（0,1）
    bestFeatureId=-1
    #循环特征列
    for i in range(numFeatures):  #for i in (0,1)
        #取出每个特征所有可能的取值  去重
        list1=[ line[i] for line in dataset]
        uniqueValues=set( list1 )  #每一列去重后的值 
        newEnt=0.0 #条件熵
        for value in uniqueValues: #取值数目越多，即len(uniqueValues) 越大
            #调用splitDataSet (列 ，set的值)  -》 得到子集
            subDataSet=splitDataSet(dataset,i,value)
            #计算概率
            prob=len(subDataSet)/float(len(dataset))
            #计算熵
            newEnt+=prob*calcShannonEnt(subDataSet)
        print('第',i,'列信息熵为：',newEnt)
        gain=ent-newEnt
        print('第',i,'列信息熵增yi为：',gain)
        
        if(gain>bestGain):
            bestGain=gain
            bestFeatureId=i
    #计算增 yi
    #比较取出最佳增 yi
    return bestFeatureId

In [10]:
#测试 求出数据集中可用于切分的最好特征列
dataset,labels=createDataSet()
index=chooseBestFeatures(dataset)
print(index)

第 0 列信息熵为： 0.4512050593046014
第 0 列信息熵增yi为： 0.5487949406953986
第 1 列信息熵为： 1.0
第 1 列信息熵增yi为： 0.0
0


In [14]:
#树的创建   树是一个字典的结构
def createTree(dataset,labels):
    #求出  dataset 中的样本所属的类别  递归的终止条件之一
    classList=[example[-1]  for example in dataset]   #  ['yes', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no']
    if classList.count(classList[0] )==len(dataset):
        return classList[0]
    
    #选取最好的特征列来进行切分
    bestFeat=chooseBestFeatures(dataset)  
    if bestFeat < 0:
        return None
    
    print('本轮最优特征的编号：',bestFeat)
    #labels 中取出最优的名字
    bestFeatLabel=labels[bestFeat]
    print('本轮最优特征的特征列名：',bestFeatLabel)
    
    #穿件一个树 这里这可树用字典表示树
    myTree={bestFeatLabel:{}}
    #从labels 中删除  bestFeatLabel
    del(labels[bestFeat])
    
    #取出最优列的值
    featValues=[ example[bestFeat] for example in dataset]
    #去重 
    uniqueValue=set(featValues)
    #根据bestFeat  及 uniqueValue 每个值来切分树的节点
    for value in uniqueValue:
        subLables=labels[:]
        temp=splitDataSet(dataset,bestFeat,value)
        
        sonTree=createTree(temp,subLables)
        myTree[bestFeatLabel][value]=sonTree
    return myTree

In [15]:
dataset,labels=createDataSet()
tree=createTree(dataset,labels)
tree

第 0 列信息熵为： 0.4512050593046014
第 0 列信息熵增yi为： 0.5487949406953986
第 1 列信息熵为： 1.0
第 1 列信息熵增yi为： 0.0
本轮最优特征的编号： 0
本轮最优特征的特征列名： 是否有房产
第 0 列信息熵为： 0.5509775004326937
第 0 列信息熵增yi为： 0.17095059445466865
本轮最优特征的编号： 0
本轮最优特征的特征列名： 是否有车


{'是否有房产': {0: {'是否有车': {0: None, 1: 'no'}}, 1: 'yes'}}

In [16]:
def classNum(classList):
    '''
    classList 分类的名称
    '''
    classCount={}
    for label in classList:
        classCount[label]=classCount.get(label,0)+1
    #对字典进行排序
    sortedClassCount=sorted( classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

In [19]:
classNum(['yes', 'yes', 'yes', 'no', 'no', 'no', 'no'])

'no'

In [20]:
#改进  加入剪枝类别  类别的取值不是按照第一个类别  而是按照数量


def createTree(dataset,labels,deepth=0,max_features=None,max_deepth=None):
    #求出  dataset 中的样本所属的类别  递归的终止条件之一
    classList=[example[-1]  for example in dataset]   #  ['yes', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'no']
    
    #增加终止条件
    if classList.count(classList[0] )==len(dataset):
        return classList[0]   
       
    #终止条件二   列中的取值种类  <= max_features 时
    if max_features==None:
        max_features=1
        
    if len(dataset[0])<=max_features:
        return classNum(classList)
    
    if max_deepth!=None:
        if deepth >=max_deepth:
            return classNum(classList)
        deepth+=1
    
    #选取最好的特征列来进行切分
    bestFeat=chooseBestFeatures(dataset)  
    if bestFeat < 0:
        # return None   这个方法返回的值会有一定问题
        return classNum(classList)
    
    print('本轮最优特征的编号：',bestFeat)
    #labels 中取出最优的名字
    bestFeatLabel=labels[bestFeat]
    print('本轮最优特征的特征列名：',bestFeatLabel)
    
    #穿件一个树 这里这可树用字典表示树
    myTree={bestFeatLabel:{}}
    #从labels 中删除  bestFeatLabel
    del(labels[bestFeat])
    
    #取出最优列的值
    featValues=[ example[bestFeat] for example in dataset]
    #去重 
    uniqueValue=set(featValues)
    #根据bestFeat  及 uniqueValue 每个值来切分树的节点
    for value in uniqueValue:
        subLables=labels[:]
        temp=splitDataSet(dataset,bestFeat,value)
        
        sonTree=createTree(temp,subLables,deepth,max_features,max_deepth)
        myTree[bestFeatLabel][value]=sonTree
    return myTree

In [25]:
dataset,labels=createDataSet()
tree=createTree(dataset,labels,max_features=0,max_deepth=2)
tree

第 0 列信息熵为： 0.4512050593046014
第 0 列信息熵增yi为： 0.5487949406953986
第 1 列信息熵为： 1.0
第 1 列信息熵增yi为： 0.0
本轮最优特征的编号： 0
本轮最优特征的特征列名： 是否有房产
第 0 列信息熵为： 0.5509775004326937
第 0 列信息熵增yi为： 0.17095059445466865
本轮最优特征的编号： 0
本轮最优特征的特征列名： 是否有车


{'是否有房产': {0: {'是否有车': {0: 'no', 1: 'no'}}, 1: 'yes'}}

In [28]:
def testClassify(tree,labels,testData):   #  0,1
    #从树中找到第一个节点 看他是哪个列
    firstkey=list(tree.keys())[0]
    secondDict=tree[firstkey]
    bestFeatIndex=labels.index(firstkey)
    
    #从testData 的这个列取值 1
    value=testData[bestFeatIndex]
    #再树中比对
    sonTreeValue=secondDict[value]
    if isinstance(sonTreeValue,dict):
        classLable=testClassify(sonTreeValue,labels,testData)
    else:
        classLable=sonTreeValue
    return classLable

In [30]:
dataset,labels=createDataSet()
testClassify(tree,labels,[1,1])

'yes'

In [32]:
#使用 以上代码测试 实现预测隐形眼镜  
fr=open('lenses.txt')
lenses=[line.strip().split('\t')  for line in fr.readlines() ]
lensesLabe=['age','presecript','astigmatic','tearRate']
tree=createTree(lenses,lensesLabe)
print(tree)

第 0 列信息熵为： 1.286691021718177
第 0 列信息熵增yi为： 0.03939650364612124
第 1 列信息熵为： 1.2865766899407325
第 1 列信息熵增yi为： 0.039510835423565815
第 2 列信息熵为： 0.9490822953528211
第 2 列信息熵增yi为： 0.37700523001147723
第 3 列信息熵为： 0.7772925846688997
第 3 列信息熵增yi为： 0.5487949406953986
本轮最优特征的编号： 3
本轮最优特征的特征列名： tearRate
第 0 列信息熵为： 1.3333333333333333
第 0 列信息熵增yi为： 0.22125183600446618
第 1 列信息熵为： 1.4591479170272446
第 1 列信息熵增yi为： 0.09543725231055489
第 2 列信息熵为： 0.7841591278514218
第 2 列信息熵增yi为： 0.7704260414863776
本轮最优特征的编号： 2
本轮最优特征的特征列名： astigmatic
第 0 列信息熵为： 0.6666666666666666
第 0 列信息熵增yi为： 0.2516291673878229
第 1 列信息熵为： 0.4591479170272448
第 1 列信息熵增yi为： 0.4591479170272448
本轮最优特征的编号： 1
本轮最优特征的特征列名： presecript
第 0 列信息熵为： 0.0
第 0 列信息熵增yi为： 0.9182958340544896
本轮最优特征的编号： 0
本轮最优特征的特征列名： age
第 0 列信息熵为： 0.3333333333333333
第 0 列信息熵增yi为： 0.3166890883150208
第 1 列信息熵为： 0.4591479170272448
第 1 列信息熵增yi为： 0.19087450462110933
本轮最优特征的编号： 0
本轮最优特征的特征列名： age
第 0 列信息熵为： 0.0
第 0 列信息熵增yi为： 1.0
本轮最优特征的编号： 0
本轮最优特征的特征列名： presecript
{'tearRate': {

In [33]:
lensesLabel=['age','presecript','astigmatic','tearRate']
testClassify(tree,lensesLabel,['presbyopic','myope','yes','normal'])

'hard'